In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import missingno as ms
import matplotlib.pyplot as plt
from src.loan_pred.preprocessing.preprocess import convert_dtype

In [2]:
train_prevloans = pd.read_csv("../data/preprocessed/train/train_prevloans.csv")
train_dg = pd.read_csv("../data/preprocessed/train/train_dg.csv")
train_perf = pd.read_csv("../data/preprocessed/train/train_perf.csv")

In [3]:
print(f"Shape of train_dg: {train_dg.shape}")
print(f"Shape of train_perf: {train_perf.shape}")
print(f"Shape of train_loans: {train_prevloans.shape}")

Shape of train_dg: (4346, 8)
Shape of train_perf: (4368, 7)
Shape of train_loans: (18183, 8)


# Graph method approaches

prevLoans Graph
demographic Graph
loans Graph

relation: loans graph --> has previous loans --> prevLoans
relation: loans_graph --> is located --> demographic Graph

In [4]:
import dgl

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


In [6]:
all_users = list(dict.fromkeys(train_perf.customerid + train_dg.customerid + train_prevloans.customerid))

In [28]:
merge1 = pd.merge(train_perf, train_dg, on="customerid", how="left", suffixes=("_perf","_dg"))
merge2 = pd.merge(merge1,train_prevloans, on="customerid", how="left", suffixes=("_perf", "_prev"))

In [29]:
merge1.shape

(4376, 14)

In [30]:
merge2.shape

(18212, 21)

In [31]:
merge2.isnull().sum()

customerid                          0
loannumber_perf                     0
approveddate                        0
loanamount_perf                     0
totaldue_perf                       0
termdays_perf                       0
good_bad_flag                       0
birthdate                        4514
bank_account_type                4514
longitude_gps                    4514
latitude_gps                     4514
bank_name_clients                4514
employment_status_clients        4514
is_missing_emp_status_clients    4514
loannumber_prev                     9
loanamount_prev                     9
totaldue_prev                       9
termdays_prev                       9
closeddate_days                     9
firstduedate_days                   9
firstrepaiddate_days                9
dtype: int64

In [18]:
merge2.head()

,customerid,loannumber_perf,approveddate,loanamount_perf,totaldue_perf,termdays_perf,good_bad_flag,birthdate,bank_account_type,longitude_gps,...,bank_name_clients,employment_status_clients,is_missing_emp_status_clients,loannumber_prev,loanamount_prev,totaldue_prev,termdays_prev,closeddate_days,firstduedate_days,firstrepaiddate_days
0,8a2a81a74ce8c05d014cfb32a0da1049,12,2017-07-25 08:22:56,30000.0,34500.0,30,Good,1972-01-15,Other,3.43201,...,Diamond Bank,Permanent,False,2,10000.0,13000.0,30,16,29,16
1,8a2a81a74ce8c05d014cfb32a0da1049,12,2017-07-25 08:22:56,30000.0,34500.0,30,Good,1972-01-15,Other,3.43201,...,Diamond Bank,Permanent,False,9,10000.0,13000.0,30,29,31,27
2,8a2a81a74ce8c05d014cfb32a0da1049,12,2017-07-25 08:22:56,30000.0,34500.0,30,Good,1972-01-15,Other,3.43201,...,Diamond Bank,Permanent,False,8,20000.0,23800.0,30,52,29,52
3,8a2a81a74ce8c05d014cfb32a0da1049,12,2017-07-25 08:22:56,30000.0,34500.0,30,Good,1972-01-15,Other,3.43201,...,Diamond Bank,Permanent,False,10,20000.0,24500.0,30,24,31,24
4,8a2a81a74ce8c05d014cfb32a0da1049,12,2017-07-25 08:22:56,30000.0,34500.0,30,Good,1972-01-15,Other,3.43201,...,Diamond Bank,Permanent,False,11,20000.0,24500.0,30,26,32,26


In [22]:
all_nodes = list(train_perf.customerid)

In [ ]:
for i in all_nodes:


In [ ]:
list_graph = dgl.heterograph()

In [ ]:
def create_graph_data(perf, prev, dg, all_nodes):

In [23]:
all_nodes[0]8a2a81a74ce8c05d014cfb32a0da1049

'8a2a81a74ce8c05d014cfb32a0da1049'

In [ ]:
train_dg[]

In [24]:
merge2.shape

(13693, 21)

In [25]:
len(all_nodes)

4368

In [26]:
len(train_perf.customerid)

4368

In [32]:
nodes = train_perf.customerid.unique()

In [101]:
graphs = [
    {
        "label": train_perf.loc[train_perf.customerid == n, "good_bad_flag"].values[0],
        "node_perf_feat": train_perf.loc[train_perf.customerid == n, :].values.tolist(),
        "node_prev_feat": train_prevloans.loc[train_prevloans.customerid == n, :].values.tolist(),
        "node_dg_feat": train_dg.loc[train_dg.customerid == n, :].values.tolist()
    }
    for n in nodes if n =='8a8589f35451855401546b0738c42524'
]

In [103]:
def generate_graph(perf, prev, dg):
    nodes_list = perf.customerid
    for n in nodes_list:
        yield {
            "graph_label": perf.loc[perf.customerid == n, "good_bad_flag"].values[0],
            "node_type_perf": perf.loc[perf.customerid == n, :].values.tolist(),
            "node_type_prev": prev.loc[prev.customerid == n, :].values.tolist(),
            "node_type_dg": dg.loc[dg.customerid == n, :].values.tolist()
        }

In [90]:
generator = generate_graph(perf=train_perf, prev=train_prevloans, dg=train_dg)

In [91]:
next(generator)

{'label': 'Good',
 'node_perf_feat': [['8a2a81a74ce8c05d014cfb32a0da1049',
   12,
   '2017-07-25 08:22:56',
   30000.0,
   34500.0,
   30,
   'Good']],
 'node_prev_feat': [['8a2a81a74ce8c05d014cfb32a0da1049',
   2,
   10000.0,
   13000.0,
   30,
   16,
   29,
   16],
  ['8a2a81a74ce8c05d014cfb32a0da1049', 9, 10000.0, 13000.0, 30, 29, 31, 27],
  ['8a2a81a74ce8c05d014cfb32a0da1049', 8, 20000.0, 23800.0, 30, 52, 29, 52],
  ['8a2a81a74ce8c05d014cfb32a0da1049', 10, 20000.0, 24500.0, 30, 24, 31, 24],
  ['8a2a81a74ce8c05d014cfb32a0da1049', 11, 20000.0, 24500.0, 30, 26, 32, 26],
  ['8a2a81a74ce8c05d014cfb32a0da1049', 6, 20000.0, 23800.0, 30, 68, 29, 68],
  ['8a2a81a74ce8c05d014cfb32a0da1049', 4, 20000.0, 24500.0, 30, 30, 30, 30],
  ['8a2a81a74ce8c05d014cfb32a0da1049', 7, 20000.0, 23800.0, 30, 1, 29, 1],
  ['8a2a81a74ce8c05d014cfb32a0da1049', 1, 10000.0, 13000.0, 30, 5, 29, 5],
  ['8a2a81a74ce8c05d014cfb32a0da1049', 3, 20000.0, 24500.0, 30, 29, 29, 29],
  ['8a2a81a74ce8c05d014cfb32a0da1049', 5,

In [102]:
graphs

[{'label': 'Good',
  'node_perf_feat': [['8a8589f35451855401546b0738c42524',
    8,
    '2017-07-19 21:46:24',
    30000.0,
    39000.0,
    60,
    'Good']],
  'node_prev_feat': [['8a8589f35451855401546b0738c42524',
    6,
    20000.0,
    24500.0,
    30,
    32,
    31,
    32],
   ['8a8589f35451855401546b0738c42524', 1, 10000.0, 13000.0, 30, 24, 29, 24],
   ['8a8589f35451855401546b0738c42524', 2, 10000.0, 13000.0, 30, 26, 30, 26],
   ['8a8589f35451855401546b0738c42524', 4, 10000.0, 13000.0, 30, 5, 29, 5],
   ['8a8589f35451855401546b0738c42524', 5, 20000.0, 23800.0, 30, 30, 29, 30],
   ['8a8589f35451855401546b0738c42524', 3, 10000.0, 13000.0, 30, 27, 31, 27],
   ['8a8589f35451855401546b0738c42524', 7, 20000.0, 24500.0, 30, 30, 29, 30]],
  'node_dg_feat': []}]

In [99]:
len(test.customerid.unique())

1099

In [100]:
test.customerid.unique()[0]

'8a8589f35451855401546b0738c42524'

In [ ]:
'8a8589f35451855401546b0738c42524'